# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Novas Tecnologias em Banco de Dados</b></center></font>
  
<font size="3"><center>Prof. Dr. Sahudy Montenegro González</center></font>

## <center>Projeto Prático - CONAB COTAÇÕES</center>

**Nome**: Mateus Tsuyoshi Matsuo Hashimoto e Pedro Gonçalves Correia

**RA**: 813500 e 813281


---
### Web Scraping

Nesta seção, será realizada a implementação do código referente ao web scraping do site da Companhia Nacional de Abastecimento (CONAB) a fim de extrair os dados a respeito das cotações dos insumos agropecuários.

Como o site possui um captcha de imagem para acessar as tabelas contendo os dados das consultas, o que impossibilitou o web scraping por meio do WebDriver. Porém, a partir de algumas análises, o grupo descobriu uma forma de extrair os dados do site. Essa maneira consiste em utilizar uma url base e, a partir dela, gerar as urls para todas as consultas e, assim, conseguir extrair todos os dados do site. 

Após realizar uma consulta manualmente, o grupo verificou que o link da página possuía o seguinte formato: https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-6983528-p=1&uf=&anoFinal=2024&ano=2020&method=acaoListarConsulta&idSubGrupo=28&btnConsultar=Consultar&jcaptcha=WBLDS&idGrupo=9

Sendo:
* <b>"https://consultaweb.conab.gov.br/consultas/consultaInsumo.do"</b>: a url base;
* <b>"p=1"</b>: a paginação da tabela;
* <b>"uf="</b>: a UF selecionada para a consulta;
* <b>"anoFinal=2024"</b>: o ano final do intervalo de tempo da consulta;
* <b>"ano=2020"</b>: o ano inicial do intervalo de tempo da consulta;
* <b>"method=acaoListarConsulta"</b>: o método do sistema para listar os dados da consulta;
* <b>"idSubGrupo=28"</b>: o id do SubGrupo selecionado para a consulta;
* <b>"btnConsultar=Consultar"</b>: o botão de confirmar a consulta;
* <b>"jcaptcha=WBLDS"</b>: o captcha utilizado para realizar a consulta;
* <b>"idGrupo=9"</b>: o id do Grupo selecionado para a consulta.


Com base nisso, o grupo alterou o link manualmente para verificar se seria possível realizar uma consulta diferente com o mesmo captcha, a fim de tentar solucionar essa questão impeditiva até então. Felizmente, foi possível concluir essa consulta graças à uma falha de segurança no sistema do site. A partir disso, a equipe realizou o código do web scraping a seguir.  

Primeiramente, vamos realizar a importação das bibliotecas que serão utilizadas no projeto.

In [1]:
import requests  # Realiza requisições HTTP para acessar páginas web
from bs4 import BeautifulSoup  # Faz parsing do HTML e extração de informações de páginas web
import pandas as pd  # Manipula e analisa dados, especialmente em dataframes
import time  # Permite controlar o tempo, inserindo delays entre requisições

Agora, vamos estabelecer a url base e os parâmetros fixos.

In [2]:
# url base
url = "https://consultaweb.conab.gov.br/consultas/consultaInsumo.do"

# Parâmetros fixos
ANO_INICIAL = 2020
ANO_FINAL = 2024
JCAPTCHA = "WBLDS"

A seguir, criaremos 3 dicionários fundamentais para realizar o web scraping desse site.

O primeiro dicionário serve para associar os grupos com os seus subgrupos por meio do id, para poder realizar todas as consultas possíveis.

In [3]:
grupos_subgrupos = {
    9: [28, 29, 30, 31, 33, 35],
    27: [32, 70, 71],
    31: [127, 128, 129],
    57: [137, 319, 339],
    39: [139, 140, 322]
}

O segundo e o terceiro dicionário servem para mapear os ids dos grupos para os seus nomes e os ids dos subgrupos para os seus nomes, respectivamente. Esses dicionários têm como objetivo armazenar os nomes dos grupos e subgrupos, uma vez que as tabelas resultantes das consultas só possuem os nomes dos produtos. 

In [4]:
# Dicionário para mapear os IDs para os nomes dos grupos
grupos_nomes = {
    9: "Agrotóxico",
    27: "Fertilizante",
    31: "Implemento",
    57: "Máquinas/Motores",
    39: "Material Propagativo"
}

# Dicionário para mapear os IDs para os nomes dos subgrupos
subgrupos_nomes = {
    28: "Acaricida",
    29: "Espalhante/Adjuvante",
    30: "Fungicida",
    31: "Herbicida",
    33: "Inseticida",
    35: "Estimulante/Regulador de Crescimento",
    32: "Inoculante",
    70: "Orgânico",
    71: "Químico",
    127: "Colheitadeira",
    128: "Manual",
    129: "Mecânico",
    137: "Máquina Agrícola",
    319: "Conjunto",
    339: "Máquina",
    139: "Mudas",
    140: "Sementes",
    322: "Maniva"
}

Agora, vamos criar duas listas, uma para armazenar os dados coletados e outra para pegar os cabeçalhos da tabela.

In [5]:
data = [] # Lista para armazenar os dados coletados
headers = [] # Lista para armazenar os titulos das colunas

Em seguida, iremos extrair todos os dados das tabelas geradas a partir das consultas acessadas por meio da construção das urls.

In [6]:
# Loop para cada combinação de grupo e subgrupo
for id_grupo, subgrupos in grupos_subgrupos.items():
    for id_subgrupo in subgrupos:
        pagina = 1  # Começa na primeira página
        
        while True:
            # Parâmetros da requisição
            parametros = {
                "d-6983528-p": pagina,
                "uf": "",
                "anoFinal": ANO_FINAL,
                "ano": ANO_INICIAL,
                "method": "acaoListarConsulta",
                "idSubGrupo": id_subgrupo,
                "btnConsultar": "Consultar",
                "jcaptcha": JCAPTCHA,
                "idGrupo": id_grupo    
            }
            
            resposta = requests.get(url, params=parametros) # Fazendo a requisição
            
            print(resposta.url)
            
            # Verifica se a requisição foi bem-sucedida
            if resposta.status_code != 200:
                print(f"Erro ao acessar {resposta.url}: {resposta.status_code}")
                break
            
            soup = BeautifulSoup(resposta.text, 'html.parser') # Parseando a página
            table = soup.find('table', {'id': 'tabelaInsumo'}) # Encontra a tabela
            
            # Se não há dados, paramos a iteração
            if table is None:
                break
            
            # Na primeira iteração, extrai os headers
            if pagina == 1 and not data:
                for i in table.find_all('th'):
                    title = i.text.strip()
                    headers.append(title)
                headers.extend(['Grupo', 'Subgrupo']) # Acrescenta os headers de grupo e subgrupo
            
            page_data = [] # Lista para armazenar os dados da página
            
            # Extraindo dados da tabela
            for row in table.find_all('tr')[1:]:
                cols = row.find_all('td')
                row_data = [col.text.replace('\n', '').strip() for col in cols]
                grupo_valor = grupos_nomes.get(id_grupo) # Pega o valor(nome) do grupo pelo id_grupo
                subgrupo_valor = subgrupos_nomes.get(id_subgrupo) # Pega o valor(nome) do subgrupo pelo id_subgrupo
                row_data.extend([grupo_valor, subgrupo_valor]) # Adiciona o valor(nome) do grupo e subgrupo
                page_data.append(row_data)
            
            data.extend(page_data) # Armazena os dados coletados
            
            print(f"Grupo {id_grupo} | Subgrupo {id_subgrupo} | Página {pagina} | {len(page_data)} registros coletados.")
            
            pagina += 1 # Próxima página
            
            time.sleep(1) # Adiciona um pequeno delay para evitar sobrecarga no servidor

https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-6983528-p=1&uf=&anoFinal=2024&ano=2020&method=acaoListarConsulta&idSubGrupo=28&btnConsultar=Consultar&jcaptcha=WBLDS&idGrupo=9
Grupo 9 | Subgrupo 28 | Página 1 | 75 registros coletados.
https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-6983528-p=2&uf=&anoFinal=2024&ano=2020&method=acaoListarConsulta&idSubGrupo=28&btnConsultar=Consultar&jcaptcha=WBLDS&idGrupo=9
Grupo 9 | Subgrupo 28 | Página 2 | 32 registros coletados.
https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-6983528-p=3&uf=&anoFinal=2024&ano=2020&method=acaoListarConsulta&idSubGrupo=28&btnConsultar=Consultar&jcaptcha=WBLDS&idGrupo=9
https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-6983528-p=1&uf=&anoFinal=2024&ano=2020&method=acaoListarConsulta&idSubGrupo=29&btnConsultar=Consultar&jcaptcha=WBLDS&idGrupo=9
Grupo 9 | Subgrupo 29 | Página 1 | 75 registros coletados.
https://consultaweb.conab.gov.br/consultas/consultaInsumo.do?d-

Após a extração de todos os dados do site, vamos converte-los para um dataframe e salvá-los em um arquivo CSV.

In [24]:
df = pd.DataFrame(data, columns=headers)

In [8]:
df.to_csv('insumos_agropecuarios.csv', index=False)

---
### Limpeza e Pré-processamento dos dados

Nesta seção, vamos realizar a preparação dos dados extraídos pelo web scraping a fim de assegurar a qualidade deles.

Primeiro, iremos analisar algumas características desse conjunto de dados.

In [25]:
df.head()

,Produto,Unidade,UF,Ano,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,Grupo,Subgrupo
0,"DANIMEN, 300 EC",L,RO,2020,"0,00","0,00","178,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","175,00","0,00",Agrotóxico,Acaricida
1,ENVIDOR,L,SP,2020,"0,00","0,00","525,00","525,00","0,00","0,00","0,00","0,00","0,00","0,00","468,17","0,00",Agrotóxico,Acaricida
2,"KUMULUS, 800 G/KG, ENXOFRE, BASF S.A., BENTLEY...",KG,SP,2020,"0,00","0,00","11,03","11,03","0,00","0,00","0,00","0,00","0,00","0,00","10,88","0,00",Agrotóxico,Acaricida
3,"OBERON, 240 G/L, ESPIROMESIFENO, BAYER S.A., 5...",L,BA,2020,"213,99","213,99","213,99","0,00","213,99","213,99","213,99","213,99","213,00","213,00","250,00","250,00",Agrotóxico,Acaricida
4,"OMITE, 300 G/KG, PROPARGITO, CROMPTON LTDA., R...",L,MG,2020,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","95,85","0,00",Agrotóxico,Acaricida


In [26]:
df.shape

(10677, 18)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10677 entries, 0 to 10676
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Produto   10677 non-null  object
 1   Unidade   10677 non-null  object
 2   UF        10677 non-null  object
 3   Ano       10677 non-null  object
 4   Jan       10677 non-null  object
 5   Fev       10677 non-null  object
 6   Mar       10677 non-null  object
 7   Abr       10677 non-null  object
 8   Mai       10677 non-null  object
 9   Jun       10677 non-null  object
 10  Jul       10677 non-null  object
 11  Ago       10677 non-null  object
 12  Set       10677 non-null  object
 13  Out       10677 non-null  object
 14  Nov       10677 non-null  object
 15  Dez       10677 non-null  object
 16  Grupo     10677 non-null  object
 17  Subgrupo  10677 non-null  object
dtypes: object(18)
memory usage: 1.5+ MB


A seguir, vamos transformar algumas variáveis categóricas em numéricas com o intuito de analisar os valores estatísticos delas.

In [28]:
cat = ['Ano', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'] # Lista de colunas a serem convertidas para numéricas

for column in cat:
    df[column] = pd.to_numeric(df[column].astype(str).str.replace(',', '.'), errors='coerce')

Após isso, criaremos uma nova variável para armazenar a região do país de cada cotação a partir de sua UF.

Para isso, será necessário criar um dicionário para o mapeamento de UF para região e depois, utilizar o método map( ) para criar a nova variável.

In [29]:
# Dicionário para mapear as regiões a partir das UFs
regioes = {
    'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'Centro-Oeste': ['GO', 'MT', 'MS', 'DF'],
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
    'Sul': ['PR', 'RS', 'SC']
}

uf_regiao = {uf: regiao for regiao, ufs in regioes.items() for uf in ufs} # Inverte o dicionário de regiões: para cada UF, associa-se sua região
df['Regiao'] = df['UF'].map(uf_regiao)

Agora, vamos reorganizar as colunas do dataframe para uma melhor visualização dos dados.

In [30]:
new_order = ['Grupo', 'Subgrupo', 'Produto', 'Unidade', 'Regiao', 'UF', 'Ano', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'] # Lista com a nova ordem das colunas

df = df[new_order]

Para fins analíticos dos dados coletados, não iremos realizar o tratamento de valores nulos, pois vamos analisar se teve ou não produção de um determinado produto em um determinado mês/ano. Portanto, manter esses valores é de extrema importância para o objetivo do negócio.

---
### Integração com o banco de dados do PostgreSQL

Nesta seção, iremos conectar ao banco de dados do PostgreSQL para armazenar os dados coletados e pré-processados.

Primeiro, vamos importar a biblioteca SQLAlchemy para fazer essa conexão com o banco.

In [33]:
from sqlalchemy import create_engine